In [ ]:
import numpy as np 
import pandas as pd
import os

In [ ]:
!nvidia-smi

In [ ]:
!cat /proc/cpuinfo | grep name | cut -f2 -d: | uniq -c

In [ ]:
!getconf LONG_BIT

In [ ]:
import argparse
import urllib

from fastai.vision.all import accuracy
from fastai.vision.all import aug_transforms
from fastai.vision.all import ImageDataLoaders
from fastai.vision.all import Learner
from fastai.vision.all import SimpleCNN
from fastai.vision.all import untar_data
from fastai.vision.all import URLs
import optuna

from optuna.integration import FastAIPruningCallback

from pathlib import Path
from fastai.vision.all import *
import pandas as pd
import optuna
import pprint

from fastai.optimizer import *
from fastai.data import *
from fastai.optimizer import *
from fastai.vision.all import accuracy

In [ ]:
optuna.visualization.matplotlib.is_available()

In [ ]:
#path = Path('../input/caltech256/256_ObjectCategories')
#path = Path('../input/stanford-dogs-dataset/images')
path = Path('../input/caltech101/101_ObjectCategories')

In [ ]:
dls = ImageDataLoaders.from_folder(
    path, valid_pct=0.25, item_tfms=Resize(224)
)
dls.show_batch()

In [ ]:
####更改模型
learn = cnn_learner(dls, resnet50, pretrained = True,metrics=[accuracy],model_dir="/tmp/model/")

In [ ]:
lr_min,lr_steep=learn.lr_find(suggestions = True)
lr_min,lr_steep

In [ ]:
#learn.unfreeze()
#learn.freeze_to(-2)
#lr_min,unfrozen_lr_steep = learn.lr_find(suggestions=True)
#lr_min,unfrozen_lr_steep

In [ ]:
import argparse
import urllib

from fastai.vision.all import accuracy
from fastai.vision.all import aug_transforms
from fastai.vision.all import ImageDataLoaders
from fastai.vision.all import Learner
from fastai.vision.all import SimpleCNN
from fastai.vision.all import untar_data
from fastai.vision.all import URLs

import optuna
from optuna.integration import FastAIPruningCallback

# TODO(crcrpar): Remove the below three lines once fastai gets compatible with torchvision v0.9.
# Register a global custom opener to avoid HTTP Error 403: Forbidden when downloading MNIST.
# This is a temporary fix until torchvision v0.9 is released.
opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)


#BATCHSIZE = 128
#EPOCHS = 30
epochs  = 30
#unfrozen_epochs = 30

#path = untar_data(URLs.MNIST_SAMPLE)


def objective(trial):
    # Data Augmentation
    apply_tfms = trial.suggest_categorical("apply_tfms", [True, False])
    if apply_tfms:
        # MNIST is a hand-written digit dataset. Thus horizontal and vertical flipping are
        # disabled. However, the two flipping will be important when the dataset is CIFAR or
        # ImageNet.
        tfms = aug_transforms(
            do_flip = False,
            flip_vert = False,
            max_rotate = trial.suggest_int("max_rotate", 0, 45),
            max_zoom = trial.suggest_float("max_zoom", 1, 2),
            p_affine = trial.suggest_float("p_affine", 0.1, 1.0, step=0.1),
            max_lighting = trial.suggest_uniform("max_lighting", 0.0, 1.0),
        )
        
    ######网络参数空间
    opt_func = trial.suggest_categorical("opt_func", [Adam, RAdam, SGD, ranger]) ###优化器
    concat_pool = trial.suggest_categorical("concat_pool", [True, False])   ###concat_pool   or 平均池化
    first_bn = trial.suggest_categorical("first_bn", [True, False])
    bn_final = trial.suggest_categorical("bn_final", [True, False])
    #lin_ftrs = trial.suggest_categorical("lin_ftrs", [[128], [256], [512], [1024]])  ###全连接层神经元数
    #lin_ftrs = trial.suggest_categorical("lin_ftrs", [128, 256, 512, 1024])  ###全连接层神经元数
    #ps = trial.suggest_discrete_uniform("ps", 0.1, 1.0, 0.1)   ##dropout
    ps = []
    lin_ftrs = []
    ps_1 = trial.suggest_discrete_uniform("ps_1", 0.1, 1.0, 0.1)
    ps.append(ps_1)
    n_layers = trial.suggest_int("n_layers", 1, 3)
    for i in range(n_layers):
        out_lin_ftrs = trial.suggest_categorical("lin_ftrs_{}".format(i), [128, 256, 512, 1024])
        lin_ftrs.append(out_lin_ftrs)
        out_ps = trial.suggest_discrete_uniform("ps_{}".format(i + 1), 0.1, 1.0, 0.1)
        ps.append(out_ps)
    
    #####训练空间
    bs = trial.suggest_categorical("bs", [64, 128, 256]) 
    #epochs = trial.suggest_int("epochs", 1, 20)
    base_lr = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    #unfrozen_epochs = trial.suggest_int("unfrozen_epochs", 1, 20)
    #unfrozen_lr = trial.suggest_loguniform("unfrozen_learning_rate", 1e-6, 1e-1)
    
    data = ImageDataLoaders.from_folder(
        path, bs = bs, valid_pct = 0.25, item_tfms = Resize(224),batch_tfms = tfms if apply_tfms else None
    )
    ##修改模型类型
    learn = cnn_learner(
        dls, resnet50, pretrained = True, opt_func = opt_func, concat_pool = concat_pool,
        first_bn = first_bn, bn_final = bn_final, lin_ftrs = lin_ftrs, ps = ps, metrics = [accuracy],
    )

    # See https://forums.fast.ai/t/how-to-diable-progress-bar-completely/65249/3
    # to disable progress bar and logging info
    learn.fit_one_cycle(
        epochs,
        lr_max = base_lr,
        cbs=[FastAIPruningCallback(trial, monitor="accuracy")],  # Monitor "error_rate"
    )
  ##learn.unfreeze()
  ##learn.freeze_to(-2)
  ##learn.fit_one_cycle(
  ##    unfrozen_epochs,
  ##    lr_max = unfrozen_lr,
  ##    cbs = [FastAIPruningCallback(trial, monitor="accuracy")],  # Monitor "accuracy"
  ##)
    t0 = time.time()
    learn.validate()
    t1 = time.time()
    execute_time = t1 - t0
    trial.set_user_attr("execute_time", execute_time)
    return learn.validate()[-1]

In [ ]:
study_name = "auto_augmentation_&&_adptive_transferlearning"  # Unique identifier of the study.
study = optuna.create_study(
    direction = "maximize",
    #load_if_exists = True,
    study_name = study_name,
    #storage = f"{output_path}/resnet50_caltech101.db",
    #pruner=optuna.pruners.SuccessiveHalvingPruner(),####异步连续减半
    pruner = optuna.pruners.HyperbandPruner(
        min_resource = 1, reduction_factor = 3      ##hyperband  超带宽
    ),
)

In [ ]:
study.enqueue_trial(
    {
        "apply_tfms": True,
        #"epochs": 5,
        "learning_rate": lr_steep,
        "unfrozen_learning_rate": unfrozen_lr_steep,
        "first_bn": True,
        "bn_final": False,
        "n_layers": 1,
        "lin_ftrs_1": 512,
        "bs": 128,
        "concat_pool": True,
        "opt_func": ranger,
        "ps_1": 0.25,
        "ps_2": 0.5,
    }
)

In [ ]:
#pruner = optuna.pruners.MedianPruner()
#study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=20, timeout=60*60*8, show_progress_bar=True)

In [ ]:
study.best_value, study.best_params

In [ ]:
optuna.visualization.matplotlib.plot_optimization_history(study)

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(study)

In [ ]:
optuna.visualization.matplotlib.plot_slice(study, params=[lin_ftrs])

In [ ]:
optuna.importance.get_param_importances(study)

In [ ]:
df = study.trials_dataframe()
df

In [ ]:
df.sort_values(['value'], ascending=False)

In [ ]:
df94 = df[df['value'] >= 0.94]

In [ ]:
#pruned_trials = study.get_trials(deepcopy = False, states = [TrialState.PRUNED])
#complete_trials = study.get_trials(deepcopy = False, states = [TrialState.COMPLETE])
#
#print("Study statistics:    ")
#print("  Number of finished trials:     ", len(study.trials))
#print("  Number of pruned trials:     ", len(pruned_trials))
#print("  Number of complete trials:     ", len(complete_trials))

In [ ]:
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))